<a href="https://colab.research.google.com/github/HannaKi/Deep_Learning_in_LangTech_course/blob/master/language_classifier_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare SVM and NN with language classification 

Course data folder language_identification contains data for 5 languages. Based on this data
* Train an SVM classifier for language recognition between these 5 languages. (Previously done!)
* Implement this same classifier using a simple NN
* Compare the results you get with NN and SVM Focus on experimenting with the various NN parameters of learning (learning rate, optimizer, etc)

In [0]:
# Reading the data in makes sense to structure a little bit
# ratkaise, miten kansio tuodaan omasta GitHubista!
# toimii myös luomalla Colabiin kansion (tässä nimeltä "texts"), 
# jonne tiedostot raahaa (kansio katoaa, kun ajo päättyy)

import random

def read_data_one_lang(lang,part):
    """Reads one file for one language. Returns data in the form of pairs of (lang,line)"""
    filename="texts/{}_{}.txt".format(lang,part)
    result=[] #this will be the list of pairs (lang,line)
    with open(filename) as f:
        for line in f:
            line=line.strip()
            result.append((lang,line)) 
    return result


def read_data_all_langs(part):
    """Reads train, test or dev data for all languages. part can be train, test, or devel"""
    data=[]
    for lang in ("en","es","et","fi","pt"):
        pairs=read_data_one_lang(lang,part)
        data.extend(pairs) #just add these lines to the end
    #...done
    #but now they come in the order of languages
    #we really must scramble these!
    random.shuffle(data)
    
    #let's yet separate the labels and lines, we will need that anyway
    labels=[label for label,line in data]
    lines=[line for label,line in data]
    return labels,lines

labels_train, lines_train=read_data_all_langs("train") # train and test data splitting already done!
labels_dev,lines_dev=read_data_all_langs("devel")


In [44]:
for label,line in zip(labels_train[:5],lines_train[:5]):
    print(label,"   ",line[:30],"...")

print(labels_train[0], lines_train[0])

pt     « Quando fui autarca e fazia r ...
es     En una de sus cacerías perdier ...
pt     Total de mortos em 92: 310 ...
pt     O cartão TVA vai permitir a se ...
en     Thanks, Jean ...
pt « Quando fui autarca e fazia realojamentos era normal que quem não conseguia logo uma habitação ficasse insatisfeito.


# Reminder

Feature matrix has row for each document

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.svm

vectorizer = CountVectorizer(max_features=100000, binary=True, ngram_range=(1,1))

feature_matrix_train = vectorizer.fit_transform(lines_train) 
# .fit_transform: Learn the vocabulary dictionary and return term-document matrix.
feature_matrix_dev = vectorizer.transform(lines_dev)
# .transform: Transform documents to document-term matrix.

# print(vectorizer.get_feature_names()) # Words (or ngrams) in learned vocabulary, a HUGE list!
print("Number of rows (documens) and unique ngrams in feature matrix")
print(feature_matrix_train.shape) 
print()
print("Since most of the texts only use a limited amonut of words (ngrams) in the vocabulary, feature matrix is sparse!")
print(feature_matrix_train.toarray())

Number of rows (documens) and unique ngrams in feature matrix
(5000, 28620)

Since most of the texts only use a limited amonut of words (ngrams) in the vocabulary, feature matrix is sparse!
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Support Vector Machine

In [46]:
for C in (0.001,0.01,0.1,1,10,100):
    classifier =  sklearn.svm.LinearSVC(C=C)
    classifier.fit(feature_matrix_train, labels_train)
    print("C=",C,"     ",classifier.score(feature_matrix_dev, labels_dev))

C= 0.001       0.8758
C= 0.01       0.9144
C= 0.1       0.933
C= 1       0.9302
C= 10       0.9102
C= 100       0.8728


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


* 93% accuracy!

# NN

For NN we will combine the data and separate a proportion of it for validation when fitting the model.

In [0]:
lines =  lines_train + lines_dev
labels = labels_train + labels_dev

For NN we need to encode each class to numeric value.
 * Remember[ difference between encoding vs. one hot encoding](https://https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f)

In [48]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder() #Turns class labels into integers
class_numbers = label_encoder.fit_transform(labels)

print(class_numbers)
print("class_numbers shape=",class_numbers.shape)
print("class labels",label_encoder.classes_) #this will let us translate back from indices to labels

[4 1 4 ... 1 4 0]
class_numbers shape= (10000,)
class labels ['en' 'es' 'et' 'fi' 'pt']


In [49]:
vectorizer = CountVectorizer(max_features=100000, binary=True, ngram_range=(1,1))

feature_matrix = vectorizer.fit_transform(lines) 
# .fit_transform: Learn the vocabulary dictionary and return term-document matrix.

# print(vectorizer.get_feature_names()) # Words (or ngrams) in learned vocabulary, a HUGE list!
print("Number of rows (documens) and unique ngrams in feature matrix")
print(feature_matrix.shape) 
print()
print("Since most of the texts only use a limited amonut of words (ngrams) in the vocabulary, feature matrix is sparse!")
print(feature_matrix.toarray())

Number of rows (documens) and unique ngrams in feature matrix
(10000, 46875)

Since most of the texts only use a limited amonut of words (ngrams) in the vocabulary, feature matrix is sparse!
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [50]:
import keras
from keras.models import Model
from keras.layers import Input, Dense

example_count, feature_count = feature_matrix.shape
class_count = len(label_encoder.classes_)

inp = Input(shape=(feature_count, )) # tuple
hidden = Dense(200, activation="tanh")(inp) # taalla kaytetty tanh. Relu suositumpi? 
# Jos mitaan aktivointifunktiota ei anneta, tulee syotteen ja kertoimien lineaarinen matriisitulo 
outp = Dense(class_count, activation="softmax")(hidden) # softmax: tuottaa luokkien jakauman
model = Model(inputs=[inp], outputs=[outp])

model

Once the model is constructed it needs to be compiled, for that we need to know:
* which optimizer we want to use (sgd is fine to begin with)
* what is the loss (categorial_crossentropy for multiclass of the kind we have is the right choice)
* which metrics to measure, accuracy is an okay choice

In [0]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [52]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

# batch_size kuinka monta inputtia kerralla sisaan. jokaisen batchin jalkeen paivitetaan painokertoimet gradientien keskiarvolla
# epochs kuinka monta kertaa mennaan lapi koko data
# validation_split: kuinka paljon dataa kaytetaan accuracyn laskemiseen

# Callback to stop training when no improvement
stop_cb=EarlyStopping(monitor='val_acc', patience=2, verbose=1, mode='auto', baseline=None, 
                      restore_best_weights=True)

hist=model.fit(feature_matrix, class_numbers, batch_size=100, verbose=1, epochs=100,
               validation_split=0.1, callbacks=[stop_cb])

Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 7s 731us/step - loss: 0.5599 - acc: 0.9226 - val_loss: 0.2110 - val_acc: 0.9590
Epoch 2/100
9000/9000 [==============================] - 5s 589us/step - loss: 0.1062 - acc: 0.9889 - val_loss: 0.1509 - val_acc: 0.9730
Epoch 3/100
9000/9000 [==============================] - 4s 470us/step - loss: 0.0468 - acc: 0.9973 - val_loss: 0.1315 - val_acc: 0.9680
Epoch 4/100
9000/9000 [==============================] - 4s 464us/step - loss: 0.0251 - acc: 0.9990 - val_loss: 0.1229 - val_acc: 0.9630
Restoring model weights from the end of the best epoch
Epoch 00004: early stopping


Experiment with the various parameters of learning (learning rate, optimizer, etc). See [Keras API](https://keras.io/optimizers/).

In [53]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.01)
model.compile(optimizer=sgd, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

hist = model.fit(feature_matrix, class_numbers, batch_size=100, verbose=1, epochs=10,
               validation_split=0.1, callbacks=[stop_cb])

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 5s 585us/step - loss: 0.0559 - acc: 0.9977 - val_loss: 0.1508 - val_acc: 0.9730
Epoch 2/10
9000/9000 [==============================] - 4s 446us/step - loss: 0.0555 - acc: 0.9977 - val_loss: 0.1506 - val_acc: 0.9730
Epoch 3/10
9000/9000 [==============================] - 4s 436us/step - loss: 0.0552 - acc: 0.9978 - val_loss: 0.1504 - val_acc: 0.9730
Restoring model weights from the end of the best epoch
Epoch 00003: early stopping


In [70]:
# import tensorflow as tf
# ops = ('Adadelta', 'Adagrad', 'Adam', 'Ftrl', 'SGD', 'Momentum', 'RMSProp', 'Adamax', 'Nadam')
# Why does the code crash with ops above? Ftrl and Momentum won't work although they are 
# documented optimizers
# and used in here: https://github.com/Jaewan-Yun/optimizer-visualization/blob/master/optimizer_visualization.py

# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Ftrl
ops = ('Adadelta', 'Adagrad', 'Adam', 'SGD', 'RMSProp', 'Adamax', 'Nadam')

for op in ops:
  print(op)
  print()
  model.compile(optimizer = op, loss="sparse_categorical_crossentropy", metrics=['accuracy'])
  hist = model.fit(feature_matrix, class_numbers, batch_size=100, verbose=1, epochs=10,
              validation_split=0.1, callbacks=[stop_cb])
  print()


Adadelta

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 6s 650us/step - loss: 8.8328e-04 - acc: 0.9998 - val_loss: 0.1177 - val_acc: 0.9610
Epoch 2/10
9000/9000 [==============================] - 4s 463us/step - loss: 8.2707e-04 - acc: 0.9998 - val_loss: 0.1232 - val_acc: 0.9580
Epoch 3/10
9000/9000 [==============================] - 4s 454us/step - loss: 7.8508e-04 - acc: 0.9998 - val_loss: 0.1263 - val_acc: 0.9560
Restoring model weights from the end of the best epoch
Epoch 00003: early stopping

Adagrad

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 6s 614us/step - loss: 0.0010 - acc: 0.9998 - val_loss: 0.1323 - val_acc: 0.9560
Epoch 2/10
9000/9000 [==============================] - 4s 422us/step - loss: 7.5296e-04 - acc: 0.9998 - val_loss: 0.1342 - val_acc: 0.9540
Epoch 3/10
9000/9000 [==============================] - 4s 428us/step - loss: 7.0974e-04 - acc: 0.9998 - 

In [77]:
lrs = np.logspace(-15, 5, base = 2, num = 20)
lrs

array([3.05175781e-05, 6.33029202e-05, 1.31309886e-04, 2.72377421e-04,
       5.64995233e-04, 1.17197531e-03, 2.43104021e-03, 5.04273124e-03,
       1.04601883e-02, 2.16976741e-02, 4.50077043e-02, 9.33599351e-02,
       1.93657455e-01, 4.01705613e-01, 8.33262006e-01, 1.72844379e+00,
       3.58532838e+00, 7.43708284e+00, 1.54268160e+01, 3.20000000e+01])

In [79]:
import numpy as np

lrs = np.logspace(-15, 5, base = 2, num = 20)

# keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
for lr in lrs:
  print("Adam, learning rate " + str(lr))
  print()
  optim = optimizers.adam(lr=lr)
  model.compile(optimizer=optim, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

  hist = model.fit(feature_matrix, class_numbers, batch_size=100, verbose=1, epochs=10,
               validation_split=0.1, callbacks=[stop_cb])
  print()

Adam, learning rate 3.0517578125e-05

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 8s 834us/step - loss: 0.3909 - acc: 0.9734 - val_loss: 1.4177 - val_acc: 0.9080
Epoch 2/10
9000/9000 [==============================] - 4s 439us/step - loss: 0.3865 - acc: 0.9739 - val_loss: 1.4158 - val_acc: 0.9080
Epoch 3/10
9000/9000 [==============================] - 4s 448us/step - loss: 0.3824 - acc: 0.9740 - val_loss: 1.4143 - val_acc: 0.9090
Epoch 4/10
9000/9000 [==============================] - 4s 438us/step - loss: 0.3787 - acc: 0.9742 - val_loss: 1.4132 - val_acc: 0.9100
Epoch 5/10
9000/9000 [==============================] - 4s 441us/step - loss: 0.3755 - acc: 0.9749 - val_loss: 1.4124 - val_acc: 0.9100
Epoch 6/10
9000/9000 [==============================] - 4s 440us/step - loss: 0.3728 - acc: 0.9753 - val_loss: 1.4118 - val_acc: 0.9110
Epoch 7/10
9000/9000 [==============================] - 4s 443us/step - loss: 0.3706 - acc: 0.9756